# Lab 2 - Automated Data Quality Assurance Pipeline
### Luke Zaruba
### GIS 5572: ArcGIS II
### 2023-03-09


In this lab, the goal is to build a pipeline that will extract data, perform QAQC operations on the data, and the save the data locally in a File Geodatabase, before saving it to a PostgreSQL database hosted on Google Cloud.

The raster pipeline was more easily applicable to both input datasets, hence why a simple function was used to check for potential issues before making any fixes. With the vector data, because they were so different, it was easiest to pull them into DataFrames and then separately start working away at cleaning them and checking them. Although it was easier to implement, it is also a less organized and messier approach, as can be seen by the cells in the Vector Pipline section.

After cleaning and repairing the data, the datasets were exported to Feature Classes or rasters within a local File Geodatabase. For the rasters, this either meant exporting it directly into the FGDB or having the final tool used to clean a dataset place the output into the FGDB. For vectors, because the cleaning was done in Pandas, the ArcGIS API for Python was used to convert the DataFrames to Spatially-Enabled DataFrames, which then allowed for easy conversion to Feature Classes from there.

Finally, the last step was to send data from the local FGDB to the GCP-hosted PostgreSQL database. This is a simple process, which can be done by creating an SDE connection and exporting the datasets from the local FGDB to the SDE.

Data Sources can be found below.

[Minnesota 30m DEM](https://gisdata.mn.gov/dataset/elev-30m-digital-elevation-model) <br>
[Minnesota NLCD 2019](https://gisdata.mn.gov/dataset/biota-landcover-nlcd-mn-2019) <br>
[BMSB Observations](https://www.eddmaps.org/distribution/viewmap.cfm?sub=9328) <br>
[MN RWIS Daily Weather Observations](https://mesonet.agron.iastate.edu/api/1/docs#/)

## Packages and Paths

In [1]:
# Packages for Processing Data
import pandas as pd
import arcpy
import arcgis

# Other Packages
import requests
import os
import warnings

warnings.filterwarnings("ignore")

In [2]:
# January Daily Weather Obs for MN Stations (GeoJSON)
weather_url = r"https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&month=1&year=2023"

# EDD Maps BMSB Data - Requested and Received via Email. (CSV path)
bmsb_path = r"C:\gitFiles\GIS5572\Lab2\Data\eddmaps_bmsb_obs.csv"

# MN NLCD 2019 Land Cover (TIF from MN Geospatial Commons)
landcover_path = r"C:\gitFiles\GIS5572\Lab2\Data\landcover\NLCD_2019_Land_Cover.tif"

# MN 30m DEM (Raster in GDB from MN Geospatial Commons)
elevation_path = r"C:\gitFiles\GIS5572\Lab2\Data\elevation\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m"

# Output FGDB Path
out_local = r"C:\Users\lukea\Documents\ArcGIS\Projects\lab2_gis5572\lab2_final.gdb" 

## Raster Pipeline

### Raster QA

In [3]:
def check_raster(file_path, categorical=True, expected_cell_size=None, expected_srid=None, xmin=None, ymin=None, xmax=None, ymax=None):
    """
    A function to check the quality of a raster dataset prior to using any methods to fix issues.
    """
    # Check for Null Values
    null_values = arcpy.management.GetRasterProperties(file_path, "ANYNODATA").getOutput(0)

    if null_values == "1":
        print("Null values exist.")
    else:
        print("Null values do not exist.")

    # Check if Cell Size is Correct
    x_size = float(arcpy.management.GetRasterProperties(file_path, "CELLSIZEX").getOutput(0))
    y_size = float(arcpy.management.GetRasterProperties(file_path, "CELLSIZEY").getOutput(0))

    if x_size == expected_cell_size and y_size == expected_cell_size:
        print("Actual spatial resolution matches expected spatial resolution.")
    else:
        print("Actual spatial resolution does not match expected spatial resolution.")

    # If Dataset is not Categorical, Check if there are Outliers
    if categorical == False:
        mean_val = float(arcpy.management.GetRasterProperties(file_path, "MEAN").getOutput(0))
        std_val = float(arcpy.management.GetRasterProperties(file_path, "STD").getOutput(0))

        max_val = float(arcpy.management.GetRasterProperties(file_path, "MAXIMUM").getOutput(0))
        min_val = float(arcpy.management.GetRasterProperties(file_path, "MINIMUM").getOutput(0))

        # Check if Min < Mean - 3 Std Devs or if Max > Mean + 3 Std Devs
        if min_val < (mean_val - (3 * std_val)) or max_val > (mean_val + (3 * std_val)):
            print("Outliers exist within the dataset. Values exist outside of +- 3 standard deviations of the mean.")
        else:
            print("Outliers do not exist within the dataset. No values +- 3 standard deviations of the mean.")
    else:
        print("Raster is categorical. Not checking for outliers.")

    # Check CRS of Raster
    sr = arcpy.Describe(file_path).spatialReference

    if expected_srid == None:
        print(f"Coordinate system of the raster is: {sr}")
    else:
        arcpy_expected_sr = arcpy.SpatialReference(expected_srid)

        if arcpy_expected_sr.factoryCode == sr.factoryCode:
            print("Actual coordinate system matches expected coordinate system.")
        else:
            print("Actual coordinate system does not match expected coordinate system.")
            print(f"Coordinate system of the raster is: {sr.factoryCode}")

    # Check if Raster is within Bounding Box (an ArcPy Polygon, with the same CS)
    if None not in [xmin, ymin, xmax, ymax]:
        left = float(arcpy.management.GetRasterProperties(file_path, "LEFT").getOutput(0))
        bottom = float(arcpy.management.GetRasterProperties(file_path, "BOTTOM").getOutput(0))
        right = float(arcpy.management.GetRasterProperties(file_path, "RIGHT").getOutput(0))
        top = float(arcpy.management.GetRasterProperties(file_path, "TOP").getOutput(0))

        if left < xmin or bottom < ymin or right > xmax or top > ymax:
            if left < xmin - 1 or bottom < ymin - 1 or right > xmax + 1 or top > ymax + 1:
                print("Raster is within 1 degree of the bounding box coordinates. Use caution and consider inspecting manually.")
            else:
                print("Raster is not completely contained within the bounding box coordinates.")
        else:
            print("Raster is completely contained within the bounding box coordinates.")
    else:
        print("Not checking bounding box.")


### Checking Rasters

In [4]:
# Running On Landcover
check_raster(landcover_path, True, 30, 26915, -97.5, 43.0, -89.00, 49.5)

Null values exist.
Actual spatial resolution matches expected spatial resolution.
Raster is categorical. Not checking for outliers.
Actual coordinate system matches expected coordinate system.
Raster is within 1 degree of the bounding box coordinates. Use caution and consider inspecting manually.


In [5]:
# Running On Elevation
check_raster(elevation_path, False, 30, 26915, -97.5, 43.0, -89.00, 49.5)

Null values exist.
Actual spatial resolution matches expected spatial resolution.
Outliers exist within the dataset. Values exist outside of +- 3 standard deviations of the mean.
Actual coordinate system matches expected coordinate system.
Raster is within 1 degree of the bounding box coordinates. Use caution and consider inspecting manually.


### Summary of Issues & Fixes

Landcover
1. Null values only exist outside of the US boundary. This is ok.
2. The raster does span into other states. **The raster should be clipped.**

Elevation
1. Null values only exist outside of the state boundary. This is ok.
2. Outliers do exist by the definition used in the function, but the values are correct. This is ok.
3. The raster is very close to the designated expected bounding box and is in the correct position. This is ok.

In [6]:
# Clipping Raster to MN BBox
arcpy.management.Clip(landcover_path, "132660 4774410 791819 5491608", os.path.join(out_local, "lc_final"));

<Result 'C:\\Users\\lukea\\Documents\\ArcGIS\\Projects\\lab2_gis5572\\lab2_final.gdb\\lc_final'>

In [7]:
# Export Elevation to Local GDB
arcpy.conversion.RasterToGeodatabase(elevation_path, out_local)

<Result ''>

## Vector Pipeline

### BMSB

In [8]:
# Loading in BMSB Observations
bmsb_df_raw = pd.read_csv(bmsb_path)

# Create Copy DF with only Certain Columns
bmsb_df = bmsb_df_raw[["objectid", "ObsDate", "Location", "Latitude", "Longitude", "NumCollect"]].copy()

# Filter where Location Contains 'Minnesota'
bmsb_df = bmsb_df[bmsb_df["Location"].str.contains("Minnesota")]

# Fill 'NumCollect' Nulls with 1
bmsb_df["NumCollect"].fillna(1, inplace=True)

# Drop Rows with Null 'Latitude' or 'Longitude'
bmsb_df = bmsb_df.dropna(subset=["Latitude", "Longitude"])

# Convert Data Types
bmsb_df["Location"] = bmsb_df["Location"].astype(str)
bmsb_df["ObsDate"] = bmsb_df["ObsDate"].astype('datetime64[ns]')
bmsb_df["NumCollect"] = bmsb_df["NumCollect"].astype(int)

# Reconfigure the Location Column to just show County Name
bmsb_df["Location"] = bmsb_df["Location"].apply(lambda x: x.replace('"', ''))
bmsb_df["County"] = bmsb_df["Location"].apply(lambda x: x.split(",")[0])
bmsb_df = bmsb_df.drop(["Location"], axis=1)

# Drop Rows where 'NumCollect' < 1
bmsb_df = bmsb_df.loc[~bmsb_df["NumCollect"] < 1]

# Drop Rows where 'NumCollect' are Outliers (> 1 Std Dev above the Mean)
numMean = bmsb_df["NumCollect"].mean()
numStd = bmsb_df["NumCollect"].std()

bmsb_df = bmsb_df.loc[~bmsb_df["NumCollect"] < numMean + numStd]

# Drop Rows where Lat/Lon are Outside MN BBox
bmsb_df = bmsb_df.loc[bmsb_df["Longitude"] > -97.5]
bmsb_df = bmsb_df.loc[bmsb_df["Longitude"] < -89.0]
bmsb_df = bmsb_df.loc[bmsb_df["Latitude"] > 43.0]
bmsb_df = bmsb_df.loc[bmsb_df["Latitude"] < 49.5]

# Result
bmsb_df

,objectid,ObsDate,Latitude,Longitude,NumCollect,County
965,3047914,2012-09-27,45.09834,-93.31717,1,Hennepin
982,3047931,2012-09-22,46.86453,-96.76807,1,Clay
1349,3048298,2011-11-16,43.67082,-92.94843,1,Mower
1513,3048462,2011-08-25,45.40731,-93.24103,1,Anoka
11094,3060908,2011-11-18,44.95374,-93.09741,1,Ramsey
...,...,...,...,...,...,...
95424,11292427,2023-01-13,44.88777,-92.97591,1,Washington
95425,11292428,2023-01-13,44.32806,-93.96222,1,Nicollet
95426,11292430,2023-01-13,44.32806,-93.96222,1,Nicollet
95427,11292431,2023-01-13,44.97945,-93.45546,1,Hennepin


In [9]:
# Convert BMSB Observations from DF to SEDF
bmsb_sedf = arcgis.GeoAccessor.from_xy(bmsb_df, "Longitude", "Latitude")

# Convert BMSB Observations from SEDF to FC
bmsb_sedf.spatial.to_featureclass(location=os.path.join(out_local, "bmsb_observations"))

'C:\\Users\\lukea\\Documents\\ArcGIS\\Projects\\lab2_gis5572\\lab2_final.gdb\\bmsb_observations'

### Weather

In [10]:
# Running On Weather Observations
weather_response = requests.get(weather_url)

weather_json = weather_response.json()["features"]

weather_df_raw = pd.DataFrame.from_records(weather_json)

# Function to Extract Fields from Dicts that are Columns in DF
def extractToCol(field):
    weather_df_raw[field] = weather_df_raw["properties"].apply(lambda x: dict(x)[field])

# Extract Fields (Properties)
weather_props = ["station", "date", "max_tmpf", "min_tmpf", "precip", "name"]

for i in weather_props:
    extractToCol(i)

# Extract Geometry
weather_df_raw["x"] = weather_df_raw["geometry"].apply(lambda x: dict(x)["coordinates"][0])
weather_df_raw["y"] = weather_df_raw["geometry"].apply(lambda x: dict(x)["coordinates"][1])

# Copy Useful Columns to new DF
weather_df = weather_df_raw[["station", "date", "max_tmpf", "min_tmpf", "precip", "name", "x", "y"]].copy()

# Fill 'Precip' Nulls with 0
weather_df["precip"].fillna(0, inplace=True)

# Drop Rows with Null 'Latitude' or 'Longitude'
weather_df = weather_df.dropna(subset=["x", "y"])

# Convert Data Types
weather_df["station"] = weather_df["station"].astype(str)
weather_df["name"] = weather_df["name"].astype(str)
weather_df["date"] = weather_df["date"].astype('datetime64[ns]')

# Drop Rows where 'precip' < 0
weather_df = weather_df.loc[weather_df["precip"] >= 0]

# Drop Outliers for Max Temp
mxtmp_mn = weather_df["max_tmpf"].mean()
mxtmp_std = weather_df["max_tmpf"].std()

weather_df = weather_df.loc[weather_df["max_tmpf"] < mxtmp_mn + (mxtmp_std * 3)]
weather_df = weather_df.loc[weather_df["max_tmpf"] > mxtmp_mn - (mxtmp_std * 3)]

# Drop Outlier for Min Temp
mntmp_mn = weather_df["min_tmpf"].mean()
mntmp_std = weather_df["min_tmpf"].std()

weather_df = weather_df.loc[weather_df["min_tmpf"] < mntmp_mn + (mntmp_std * 3)]
weather_df = weather_df.loc[weather_df["min_tmpf"] > mntmp_mn - (mntmp_std * 3)]

# Drop Outlier for Precip
precip_mn = weather_df["precip"].mean()
precip_std = weather_df["precip"].std()

weather_df = weather_df.loc[weather_df["precip"] < precip_mn + (precip_std * 3)]

# Drop Rows where Lat/Lon are Outside MN BBox
weather_df = weather_df.loc[weather_df["x"] > -97.5]
weather_df = weather_df.loc[weather_df["x"] < -89.0]
weather_df = weather_df.loc[weather_df["y"] > 43.0]
weather_df = weather_df.loc[weather_df["y"] < 49.5]

# Result
weather_df

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2023-01-01,37.040024,20.660011,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2023-01-01,26.780000,17.599989,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2023-01-01,33.440020,18.140022,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2023-01-01,33.080000,21.380000,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2023-01-01,31.819979,23.899988,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
4514,MN158,2023-01-31,13.640022,-6.879994,0.0,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171
4515,MN159,2023-01-31,4.640022,-14.079994,0.0,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050
4516,MN160,2023-01-31,7.340022,-11.200011,0.0,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311
4517,MN161,2023-01-31,7.699989,-11.020000,0.0,U.S.12 - Delano - MP 140.4 MN US MNDOT,-93.766068,45.035450


In [11]:
# Convert Weather Observations from DF to SEDF
weather_sedf = arcgis.GeoAccessor.from_xy(weather_df, "x", "y")

# Convert Weather Observations from SEDF to FC
weather_sedf.spatial.to_featureclass(location=os.path.join(out_local, "weather_observations"))

'C:\\Users\\lukea\\Documents\\ArcGIS\\Projects\\lab2_gis5572\\lab2_final.gdb\\weather_observations'

## Export from Local FGDB to PostgreSQL

In [12]:
# Set up SDE Connection using PGAdmin & Catalog Pane in ArcGIS Pro
sde = r"C:\Users\lukea\Documents\ArcGIS\Projects\lab2_gis5572\lab2.sde"

# Export Vectors to Postgres
arcpy.conversion.FeatureClassToGeodatabase(
    f'{os.path.join(out_local, "weather_observations")};{os.path.join(out_local, "bmsb_observations")}',
    sde
)

# Export Rasters to Postgres
arcpy.conversion.RasterToGeodatabase(
    f'{os.path.join(out_local, "lc_final")};{os.path.join(out_local, "digital_elevation_model_30m")}',
    sde
)

<Result ''>